# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os,sys
import pandas as pd

# Get the current working directory
current_directory = os.getcwd()
# Navigate two levels up to reach main dir
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
# Insert the specific directory into the system path
sys.path.insert(0, os.path.join(parent_directory, 'bin', 'IMBIE'))
from imbie_utils_lithk import *


# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')



### Configure IMBIE comaprision

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "AIS"# Change to "AIS" or "GIS"

# Set start and end dates
start_date = '2007-01-01'
end_date ='2014-01-01'

#Set density of ice
rho_ice = 918 # (kg/m^3)

# Select  variable for mass balance comparision
mass_balance_column="Cumulative mass balance (Gt)"# "Cumulative dynamics mass balance anomaly (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"



nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_GSFC_ISSM_historical.nc'

# Set model model path,shapefile path and projection and IMBIE csv_file
if icesheet == "GIS":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_GSFC_ISSM_historical.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    #Set the observation data dir path
    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2021_Gt.csv'


       
elif icesheet== "AIS":
    projection = "EPSG:3031"  # Antarctica    
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    #Set the observation data dir path
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
    # obs_filename = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'

else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")


### Calculate time-varying   mass balance change of IMBIE from start to end date

In [3]:
#IMBIE total mass balance
IMBIE_total_mass_change_sum=sum_MassBalance(obs_filename,start_date,end_date,mass_balance_column)

### Calculate time-varying  model mass balance change from IMBIE's start to end date 

In [4]:
modal_mass_changes=process_model_data(nc_filename,IMBIE_total_mass_change_sum,start_date, end_date,rho_ice,projection,shape_filename,icesheet)

The selected dates 2007-01-01 and 2008-01-01 are within the range of the model data.


## Calculate difference of IMBIE-model mass change

In [5]:
#Convert model mass balance data to DataFrame
model_mass_df = pd.DataFrame.from_dict(modal_mass_changes, orient='index', columns=['Model_Mass_Change'])
model_mass_df.index.name = 'Year'
model_mass_df.reset_index(inplace=True)

# Merge the two datasets on 'Year'
merged_df = pd.merge(IMBIE_total_mass_change_sum[['Year', 'IMBIE_Mass_Change']], model_mass_df, on='Year', how='inner')

# Calculate the difference
merged_df['Difference'] = merged_df['IMBIE_Mass_Change'] - merged_df['Model_Mass_Change']
#Rename Year column to Date
merged_df.rename(columns={'Year': 'Date'}, inplace=True)

In [6]:
# Display the time varying mass change
print(f'Time-varying IMBIE and model mass changes from {start_date} to {end_date}')
# Display the result
print(merged_df)

Time-varying IMBIE and model mass changes from 2007-01-01 to 2008-01-01
          Date  IMBIE_Mass_Change  Model_Mass_Change  Difference
0   2007-01-31           -20.1481           -9.19184   -10.95626
1   2007-03-02           -20.1481          -10.14272   -10.00538
2   2007-04-02           -20.1482           -9.50880   -10.63940
3   2007-05-02           -20.1481           -9.50880   -10.63930
4   2007-06-02           -20.1481           -9.50880   -10.63930
5   2007-07-02           -20.1482           -9.50880   -10.63940
6   2007-08-01           -20.1481           -9.19184   -10.95626
7   2007-09-01           -20.1481           -9.50880   -10.63930
8   2007-10-01           -20.1482           -9.50880   -10.63940
9   2007-11-01           -20.4436           -9.50880   -10.93480
10  2007-12-01           -20.4437           -9.50880   -10.93490
11  2008-01-01           -19.9333           -9.50880   -10.42450
